In [29]:

!pip install -q langchain-openai langchain playwright beautifulsoup4
!playwright install
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()
!pip install html2text

In [13]:
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

# Load HTML
loader = AsyncChromiumLoader(["https://www.wsj.com"])
html = loader.load()

In [14]:
# Transform
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["span"])

In [15]:
# Result
docs_transformed[0].page_content[0:500]

''

In [16]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.espn.com", "https://lilianweng.github.io/posts/2023-06-23-agent/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|##########| 2/2 [00:00<00:00, 20.31it/s]


In [17]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.espn.com", "https://lilianweng.github.io/posts/2023-06-23-agent/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|##########| 2/2 [00:00<00:00, 19.18it/s]


In [19]:
from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
docs_transformed[0].page_content[0:5000]

"Skip to main content  Skip to navigation\n\n<\n\n>\n\nMenu\n\n## ESPN\n\n  *   *   * scores\n\n  * NFL\n  * NBA\n  * MLB\n  * NHL\n  * Soccer\n  * Golf\n  * …\n\n    * NCAAF\n    * Sports Betting\n    * Boxing\n    * CFL\n    * NCAA\n    * Cricket\n    * F1\n    * Horse\n    * LLWS\n    * MMA\n    * NASCAR\n    * NBA G League\n    * NCAAM\n    * NCAAW\n    * Olympic Sports\n    * PLL\n    * Professional Wrestling\n    * Racing\n    * RN BB\n    * RN FB\n    * Rugby\n    * Tennis\n    * WNBA\n    * X Games\n    * UFL\n\n  * More ESPN\n  * Fantasy\n  * Watch\n  * ESPN BET\n  * ESPN+\n\n##\n\n  * Subscribe Now\n  * FA Cup: Semifinals\n\n  * NCAA Women's Gymnastics Championship\n\n  * PGA TOUR LIVE\n\n  * LPGA: Featured Groups\n\n  * UFL\n\n  * LALIGA\n\n  * Bundesliga\n\n  * MLB\n\n  * NCAA Baseball\n\n  * NCAA Softball\n\n## Quick Links\n\n  * NBA Postseason Tracker\n\n  * NFL Draft\n\n  * WNBA Draft\n\n  * NFL Free Agency\n\n  * ESPN Radio: Listen Live\n\n  * How To Watch NHL Games\n\n

In [30]:
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama


llm = Ollama(model="llama3")


In [25]:
from langchain.chains import create_extraction_chain
from langchain_community.llms import Ollama


schema = {
    "properties": {
        "news_article_title": {"type": "string"},
        "news_article_summary": {"type": "string"},
    },
    "required": ["news_article_title", "news_article_summary"],
}


def extract(content: str, schema: dict):
    return create_extraction_chain(schema, llm).extract(content)

# Extract




AttributeError: 'LLMChain' object has no attribute 'extract'

In [24]:
import pprint

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.embeddings import OllamaEmbeddings


def scrape_with_playwright(urls, schema):
    loader = AsyncChromiumLoader(urls)
    docs = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(
        docs, tags_to_extract=["span"]
    )
    print("Extracting content with LLM")

    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=0
    )
    splits = splitter.split_documents(docs_transformed)

    # Process the first split
    extracted_content = extract(schema=schema, content=splits[0].page_content)
    pprint.pprint(extracted_content)
    return extracted_content


urls = ["https://www.wsj.com"]
extracted_content = scrape_with_playwright(urls, schema=schema)

Extracting content with LLM


IndexError: list index out of range

In [27]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain_chroma import Chroma
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings



In [32]:
# Vectorstore
llm = Ollama(model="llama3")

vectorstore = Chroma(
    embedding_function=OllamaEmbeddings(), persist_directory="./chroma_db_oai"
)

# LLM
llm = Ollama(model="llama3")

# Search
search = GoogleSearchAPIWrapper()

/home/jordan/dev/LLM/HackTheRobots/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


ValidationError: 1 validation error for GoogleSearchAPIWrapper
__root__
  Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass `google_api_key` as a named parameter. (type=value_error)

In [1]:
from langchain_community.document_loaders import WikipediaLoader
docs = WikipediaLoader(query="HUNTER X HUNTER", load_max_docs=2).load()
len(docs)
docs[0].metadata  # meta-information of the Document


KeyboardInterrupt: 

In [1]:
docs[0].page_content[:400]  # a content of the Document

NameError: name 'docs' is not defined

In [2]:
#using olloma embding to get the vector of the content of the document
#using llama3 model
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
llm = Ollama(model="llama3")
vectorstore = Chroma(embedding_function=OllamaEmbeddings(), persist_directory="./chroma_db_oai")
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

# Load
docs = WikipediaLoader(query="HUNTER X HUNTER", load_max_docs=2).load()

# Vectorstore
vectorstore = Chroma(embedding_function=OllamaEmbeddings(), persist_directory="./chroma_db_oai")
vectorstore.add_documents(docs)
# Search
vectorstore.search("HUNTER X HUNTER", top_k=5) # search for the most similar documents


TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given